In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
import copy
%matplotlib inline

In [2]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-27'

In [3]:
RawDataPath=WORKSPACE_PATH+'/raw/'

In [4]:
import processing

センサデータを読み込む

In [11]:
dic1=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor1")
dic2=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor2")
dic3=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor3")
dic4=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor4")
dic5=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor5")
dic6=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor6")
dic7=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor7")

"""
data1 = processing.Dic2darray(dic1)
data2 = processing.Dic2darray(dic2)
data3 = processing.Dic2darray(dic3)
data4 = processing.Dic2darray(dic4)
data5 = processing.Dic2darray(dic5)
data6 = processing.Dic2darray(dic6)
data7 = processing.Dic2darray(dic7)
"""

'\ndata1 = processing.Dic2darray(dic1)\ndata2 = processing.Dic2darray(dic2)\ndata3 = processing.Dic2darray(dic3)\ndata4 = processing.Dic2darray(dic4)\ndata5 = processing.Dic2darray(dic5)\ndata6 = processing.Dic2darray(dic6)\ndata7 = processing.Dic2darray(dic7)\n'

windows関数を作成する。
要求仕様
* 入力データは2次元配列　raw 時間、colm　データの種類（加速度X,加速度Y,・・・）
* 出力データは4次元配列　numberOfWindow * windowWidth * kindOfData * 
# how to use
1. Set Data(1darray)
case 1  
    seed.append(AccX)  
    seed.append(AccY)  
    seed.append(AccZ)  
2. Making window
w = build(seed)  
w -> [ [AccX],[AccY],[AccZ] ]  
window = compile(w)  
window -> 
          [ [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]], ....]

In [95]:
class Window:
    """Input dictionary
       Ouput Windows
    """         
    def __init__(self):                  
        self.seed ={}
    
    def SetData(self, RegistDataName, RegistData):
        if RegistDataName in self.seed:
            print RegistDataName+' has been registed before'
            return -1
        
        self.seed[RegistDataName] = RegistData
        print RegistDataName+' is registed now'
    
    def _Build(self):
        keys = self.seed.keys()
        array = self.seed[keys[0]]
        
        for k in keys[1:]:
            array = np.vstack( (array,self.seed[k]))
        print 'Build Complete'
        return array.T
    
    def Compile(self,windowWidth,overlap):
        source = self._Build()
        offset = len(source)%windowWidth
        if offset % 2 == 1:
            offset=offset/2 +1
        else:
            offset=offset/2
        print offset
        # first array
        start = offset
        goal = start+windowWidth
        window = source[start:goal]
        
        for i in range(1,len(source/windowWidth)):
            start = int( start+windowWidth*overlap )
            goal = start+windowWidth
            adding = source[start:goal]
            window = np.vstack((window,adding))
        return window

In [96]:
w = Window()

In [97]:
w.SetData('AccX',dic1['AccX'])

AccX is registed now


In [98]:
w.SetData('AccY',dic1['AccY'])

AccY is registed now


In [99]:
w.SetData('AccZ',dic1['AccZ'])

AccZ is registed now


In [100]:
w.Compile(windowWidth=16,overlap=0.5)

Build Complete
3


array([[  2.89400000e-01,  -9.61600000e-01,  -9.54000000e-02],
       [  2.89100000e-01,  -9.63300000e-01,  -9.69000000e-02],
       [  2.92300000e-01,  -9.56800000e-01,  -9.66000000e-02],
       ..., 
       [  1.96100000e-01,  -9.71400000e-01,   5.90000000e-03],
       [  1.94200000e-01,  -9.73100000e-01,   8.00000000e-04],
       [  2.00300000e-01,  -9.76800000e-01,   6.70000000e-03]])